In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import convert_all as crd

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [2]:
#如果左右臂数据采集反了，交换前7个和后7个数据
import h5py
import os

def print_hdf5_structure(hdf5_file_path):
    with h5py.File(hdf5_file_path, 'r') as f:
        def print_attrs(name, obj):
            print(name)
        f.visititems(print_attrs)

def swap_data(hdf5_file_path):
    with h5py.File(hdf5_file_path, 'r+') as f:
        # 读取 action 和 qpos 数据
        action_data = f['/action'][:]
        qpos_data = f['observations/qpos'][:]
        
        # 交换 action 数据的前 7 个和后 7 个数据
        action_data[:, :7], action_data[:, -7:] = action_data[:, -7:], action_data[:, :7].copy()
        
        # 交换 qpos 数据的前 7 个和后 7 个数据
        qpos_data[:, :7], qpos_data[:, -7:] = qpos_data[:, -7:], qpos_data[:, :7].copy()
        
        # 将交换后的数据写回文件
        f['/action'][:] = action_data
        f['observations/qpos'][:] = qpos_data
# 文件路径模板
# file_template = '/home/qiuzhi/IMITATION_LEARNING/Imitate-All/demonstrations/hdf5/dual_put_two_cups_to_bowl/episode_{:02d}.hdf5'

# # 遍历指定范围内的所有文件
# for episode in range(30, 50):
#     hdf5_file_path = file_template.format(episode)
#     if os.path.exists(hdf5_file_path):
#         print(f"Processing {hdf5_file_path}")
#         swap_data(hdf5_file_path)
#     else:
#         print(f"File {hdf5_file_path} does not exist")
# print_hdf5_structure(hdf5_file_path)
hdf5_file_path = '/home/qiuzhi/IMITATION_LEARNING/Imitate-All/demonstrations/hdf5/dual_put_two_cups_to_bowl/episode_40.hdf5'
swap_data(hdf5_file_path)

In [4]:
# get all low_dim data
task_name = "example_task"
raw_root_dir = "../demonstrations/raw"
raw_dir = f"{raw_root_dir}/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["records.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    name_converter={
        "/obs/jq": "/observations/qpos",
        "/act": "/action",
    },
    pre_process=None
)

Data Converting:   0%|          | 0/50 [00:00<?, ?it/s]


TypeError: object of type 'int' has no len()

In [5]:
print(len(list(data.keys())))
print(data.keys())
print(data['000'].keys())
print(data['000']['/observations/qpos'][50])

NameError: name 'data' is not defined

In [ ]:
import os, cv2
# merge high_dim data and save
raw_dir
video_names = ["video.mp4"]
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
low_dim_data = data
name_converter = {"video": "/observations/images/0"}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))

max_pad_lenth = max(episode_lens)

# save all data
episode_names = list(low_dim_data.keys())
print(f"Episode lengths: {episode_lens}")
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

def save_one(index, ep_name):
    crd.merge_video_and_save(
        low_dim_data[ep_name],
        f"{raw_dir}/{ep_name}",
        video_names,
        crd.save_dict_to_hdf5,
        name_converter,
        compresser,
        f"{target_dir}/" + target_namer(index),
        max_pad_lenth,
    )
    data.pop(ep_name)

In [ ]:
# save all
from concurrent.futures import ThreadPoolExecutor
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:
    for index, ep_name in enumerate(episode_names):
        # silent execution, no print
        futures.append(executor.submit(save_one, index, ep_name))

In [ ]:
save_one(0, episode_names[0])

In [ ]:
# check hdf5 data
import convert_all as crd
task_name = "mujoco_stack_cups"
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
data = crd.hdf5_to_dict(f"{target_dir}/episode_0.hdf5")
print(data.keys())
data_flat = crd.flatten_dict(data, prefix="/")
print(data_flat.keys())

In [ ]:
print(data_flat['/compressed_len'])

In [ ]:
# show actions
start = 120
end = start + 3

action = data_flat["/action"][start:end]
obs = data_flat["/observations/qpos"][start+1:end+1]
print(action)
print(obs)
print((obs - action))

In [ ]:
print(action[1])
print(obs[1])
print((obs[1] - action[1]) * 180 / 3.1415926)

In [ ]:
import cv2
image = data_flat["/observations/images/0"][0]
print(image.shape)
compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)
image_dc = compresser.decompress(image)
print(image_dc.shape)
import matplotlib.pyplot as plt
plt.imshow(image_dc[:, :, ::-1])
plt.show()